In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
def find_recipe_links(start = 1, end = 100):
    if not isinstance(start, int) or not isinstance(start, int):
        raise Exception('Start, End 페이지가 정수 형태가 아닙니다.')

    elif start < 1:
        raise Exception('Start 페이지가 1보다 작습니다')

    url_links = []
    for i in range(start, end+1):
        url = f"https://www.10000recipe.com/recipe/list.html?order=reco&page={i+1}"
        html = requests.get(url).text
        soup = BeautifulSoup(html, "lxml")

        result = soup.select('#contents_area_full > ul > ul > li.common_sp_list_li > div.common_sp_thumb > a.common_sp_link')
        url_links += [re.attrs['href'] for re in result]

        if i % 10 == 0:
            time.sleep(0.5)
    
    return url_links

In [28]:

def find_ingredient(link):
    url = f'https://www.10000recipe.com{link}'
    html = requests.get(url).text
    soup = BeautifulSoup(html, "lxml")
    
    selectors = ['#divConfirmedMaterialArea > ul:nth-of-type(1) > a > li',
    '#divConfirmedMaterialArea > ul:nth-of-type(2) > a > li',
    '#divConfirmedMaterialArea > ul:nth-of-type(1) > li',
    
                ]

    ingredients = []
    for sel in selectors:
        try:
            result = soup.select(sel)
        except:
            continue
        for i in result:
            ingredients.append(i.contents[0].strip())

    # 메뉴가 2개 이하이거나 None일 때
    if len(ingredients) < 2:
        return None
    try:
        name = soup.select('#contents_area > div.view2_summary.st3 > h3')[0].contents[0].strip()
        image = soup.select('#main_thumbs')[0].attrs['src']
    except:
        return None
    
    return name, ingredients, image, url


In [18]:
a = find_ingredient('/recipe/6865340')
print(a)


('[간단 자취요리] 몽쉘로 쫀득쫀득 컵브라우니 만들기!', ['우유', '초콜릿', '몽쉘카카오맛'], 'https://recipe1.ezmember.co.kr/cache/recipe/2017/02/15/8fc58374cdba7cdf0beebac7f38ea2971.jpg')


In [29]:
recipe_data = []

url_links = find_recipe_links(270, 371)
print('링크 수집 완료')

for idx, link in enumerate(url_links):
    tmp = find_ingredient(link)
    if tmp != None:
        recipe_data.append(tmp)
                           
    if idx % 100 == 0:
        print(idx)


print('완료')




링크 수집 완료
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
완료


In [30]:

names = []
ingres = []
images = []
urls = []

for name, ingre, image, url in recipe_data:
    names.append(name)
    ingres.append(ingre)
    images.append(image)
    urls.append(url)


In [31]:
len(url_links)

4054

In [32]:
new_ingre = []
# names = recipe_data.keys()
# ingre = list(recipe_data.values())
# # print(ingre)
# ingre2 = []
for i in range(len(ingres)):
    new_ingre.append(','.join(ingres[i]))




In [33]:
print(len(names))
print(len(ingres))
print(len(images))
df = pd.DataFrame({'이름' : names, '재료' : new_ingre, '이미지' : images, 'URL' : urls})
df
df.to_excel('recipe_db_2.xlsx', index=False )


3977
3977
3977
